# 뷰티 + 음식 sns 언급 빈도에 따른 지역 소비금액 성장/하락률

In [900]:
import numpy as np
import pandas as pd
import glob

In [901]:
# 푸드 데이터 로드
path_food = "./data/EC_SNS_KFOOD_ATTRACTION_DATA_*.csv"
files_food = glob.glob(path_food)
df_food = [pd.read_csv(f, encoding='utf-8') for f in files_food]
sns_kfood = pd.concat(df_food, ignore_index=True)

# 뷰티 데이터 로드
path_beauty = "./data/EC_SNS_KBEAUTY_ATTRACTION_DATA_*.csv"
files_beauty = glob.glob(path_beauty)
df_beauty = [pd.read_csv(f, encoding='utf-8') for f in files_beauty]
sns_beauty = pd.concat(df_beauty, ignore_index=True)

# 데이터 병합
full_df = pd.concat([sns_kfood, sns_beauty], ignore_index=True)

In [902]:
# 채널 이름이 체널 전체인 것들만 추출 후 0으로 초기화
full_df = full_df[full_df['CHNNEL_NM'] == '채널전체'].copy()
full_df = full_df.fillna(0)

# Unnamed 컬럼 제거
full_df = full_df.loc[:, ~full_df.columns.astype(str).str.startswith("Unnamed")]

# 이상 지역 제거
full_df = full_df[full_df["SIGNGU_NM"].astype(str) != "0"]
full_df = full_df[full_df["SIGNGU_NM"].astype(str) != "KR"]
full_df = full_df[full_df["SIGNGU_NM"].astype(str) != "영동대로"]   # 도로는 3개의 구를 끼고 있어서 제거

# 사용할 컬럼 추출
use_cols = ['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM',
            'BASE_YEAR_ACCMLT_FQ_CO',       # 기준년도누적빈도수
            'TURSM_CSTMR_CO',               # 관광객 수
            'TURSM_SPND_PRICE',             # 관광소비금액 (총합)
            'AVRG_SCORE_VALUE',             # 평점값
            'REVIEW_CO',                    # 리뷰수
        ]
df_sub = full_df[use_cols].copy()

# 데이터 형 변환
df_sub['BASE_YM'] = pd.to_datetime(df_sub['BASE_YM'], format='%Y%m')
df_sub = df_sub.fillna(0)

ctprvn_map = {
    '인천': '인천광역시', 
    '서울':'서울특별시',
    '서울시': '서울특별시',
    '강원특별자치도': '강원도',
    '고양시':'경기도',
    '남양주시':'경기도',
    '부산':'부산광역시',
    '성남시':'경기도',
    '수원시': '경기도',
    '인천':'인천광역시',
    '전북특별자치도':'전라북도',
    '천안시':'강원도',
    '춘천시':'강원도'
}

signgu_map = {
    '고양시 일산동구': '고양시',
    '고양시 일산서구' : '고양시',
    '공덕역': '마포구',
    '까치산역':'강서구 ',
    '남산면':'춘천시',
    '남창동': '중구',
    '대화역':'고양시',
    '망원역' : '마포구',
    '성남시 분당구' : '성남시',
    '성북동성당' : '성북구',
    '성수1가제2동' : '성동구',
    '성신여대입구역' : '성북구',
    '수원시 영통구' : '수원시',
    '수원시 팔달구' : '수원시',
    '예지동' : '종로구',
    '용인시 기흥구' : '용인시',
    '용인시 처인구' : '용인시',
    '잠실본동' : '송파구',
    '전주시 완산구' : '전주시',
    '정자역' : '성남시',
    '천안시 동남구' :'천안시',
    '청주시 상당구' : '청주시',
    '청주시 청원구' : '청주시',
    '평내호평역' : '남양주시',
    '포항시 남구' : '포항시',
    '화양동' : '광진구'
}

df_sub['CTPRVN_NM'] = df_sub['CTPRVN_NM'].replace(ctprvn_map)
df_sub['SIGNGU_NM'] = df_sub['SIGNGU_NM'].replace(signgu_map)

# 시군구 중간 확인용
# items = sorted(df_sub['SIGNGU_NM'].unique().tolist())
# for i, item in enumerate(items):
#     print(f"{item:<12}", end="")
#     if (i + 1) % 6 == 0:       
#         print()

num_cols = [
    'TURSM_CSTMR_CO',
    'TURSM_SPND_PRICE',
    'AVRG_SCORE_VALUE',
    'REVIEW_CO',
    'BASE_YEAR_ACCMLT_FQ_CO'
]

for c in num_cols:
    df_sub[c] = pd.to_numeric(df_sub[c], errors='coerce')



In [903]:
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# 년 월 시 군구로 집계
final_df = df_sub.groupby(['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM']).agg({
    'TURSM_SPND_PRICE': 'mean',          # 관광소비금액
    'TURSM_CSTMR_CO': 'mean',            # 관광고객수
    'AVRG_SCORE_VALUE': 'mean',          # 평점값
    'BASE_YEAR_ACCMLT_FQ_CO': 'sum',     # 기준년월 sns 언급 빈도수
    'REVIEW_CO':'sum'                    # 리뷰수
}).reset_index()


# 정렬
final_df = final_df.sort_values(["CTPRVN_NM", "SIGNGU_NM", "BASE_YM"]).reset_index(drop=True)

In [904]:
# final_df.to_csv('02101236.csv')

In [905]:
# 시군구별 그룹
group_cols = ["CTPRVN_NM", "SIGNGU_NM"]
base_features = ['TURSM_SPND_PRICE', 'TURSM_CSTMR_CO', 'AVRG_SCORE_VALUE', 'BASE_YEAR_ACCMLT_FQ_CO', 'REVIEW_CO']

# 과거 값 가져오기
for col in base_features:
    final_df[f"{col}_L1"] = final_df.groupby(group_cols)[col].shift(1) # 1개월 전
    final_df[f"{col}_L2"] = final_df.groupby(group_cols)[col].shift(2) # 2개월 전

# 전월 대비 성장률 구하기
for col in base_features:
    prev = final_df.groupby(group_cols)[col].shift(1)
    final_df[f"{col}_MOM_RATE"] = (final_df[col] - prev) / prev.replace(0, np.nan)

# 최근 3개월 추세(롤링)
for col in base_features:
    final_df[f"{col}_ROLL3"] = final_df.groupby(group_cols)[col].transform(
        lambda s: s.shift(1).rolling(3, min_periods=1).mean()
    )

# 무한대/결측 정리
final_df = final_df.replace([np.inf, -np.inf], np.nan)

# 다음달 상승/하락
final_df["CSTMR_NEXT"] = final_df.groupby(group_cols)["TURSM_SPND_PRICE"].shift(-1)

# 마지막 달은 학습에서 제외
final_df["NEXT_GROWTH_LABEL"] = np.where(
    final_df["CSTMR_NEXT"].isna(),
    np.nan,
    (final_df["CSTMR_NEXT"] > final_df["TURSM_SPND_PRICE"]).astype(int)
)

In [906]:
df_all = final_df.copy()
feature_cols = final_df.columns.drop(['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM', 'CSTMR_NEXT', 'NEXT_GROWTH_LABEL']).tolist()

train_df = df_all.dropna(subset=feature_cols + ["NEXT_GROWTH_LABEL"]).copy()
pred_df  = df_all[df_all["NEXT_GROWTH_LABEL"].isna()].dropna(subset=feature_cols).copy()

X_train, X_test, y_train, y_test = train_test_split(
    train_df[feature_cols],
    train_df["NEXT_GROWTH_LABEL"],
    test_size=0.2,
    random_state=0,
    stratify=train_df["NEXT_GROWTH_LABEL"]
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

xgb_clf = XGBClassifier(
    n_estimators = 200
    , max_depth = 3
    , learning_rate = 0.1
    , random_state = 0
)

xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

# 예측
pred_df["proba_up"] = xgb_clf.predict_proba(pred_df[feature_cols])[:, 1]
pred_df["proba_down"] = 1 - pred_df["proba_up"]
pred_df["pred_label"] = (pred_df["proba_up"] >= 0.5).astype(int)

pred_df = pred_df.sort_values("proba_up", ascending=False)
# top10 = pred_df.sort_values("proba_up", ascending=False).head(10)

for _, r in pred_df.iterrows():
    label = 1 if r["proba_up"] >= 0.5 else 0
    print(f"지역 : {r['CTPRVN_NM']} {r['SIGNGU_NM']}")
    print(f"예측 결과: 인기 {'상승' if label==1 else '하락'}")
    print(f"상승 확률: {r['proba_up']:.2f}")
    print(f"하락 확률: {r['proba_down']:.2f}")
    print("-"*40)


지역 : 경기도 포천시
예측 결과: 인기 상승
상승 확률: 0.71
하락 확률: 0.29
----------------------------------------
지역 : 경상남도 하동군
예측 결과: 인기 상승
상승 확률: 0.63
하락 확률: 0.37
----------------------------------------
지역 : 경기도 가평군
예측 결과: 인기 상승
상승 확률: 0.57
하락 확률: 0.43
----------------------------------------
지역 : 강원도 춘천시
예측 결과: 인기 상승
상승 확률: 0.53
하락 확률: 0.47
----------------------------------------
지역 : 경기도 수원시
예측 결과: 인기 하락
상승 확률: 0.46
하락 확률: 0.54
----------------------------------------
지역 : 경기도 하남시
예측 결과: 인기 하락
상승 확률: 0.46
하락 확률: 0.54
----------------------------------------
지역 : 경기도 고양시
예측 결과: 인기 하락
상승 확률: 0.46
하락 확률: 0.54
----------------------------------------
지역 : 부산광역시 중구
예측 결과: 인기 하락
상승 확률: 0.46
하락 확률: 0.54
----------------------------------------
지역 : 서울특별시 관악구
예측 결과: 인기 하락
상승 확률: 0.46
하락 확률: 0.54
----------------------------------------
지역 : 서울특별시 마포구
예측 결과: 인기 하락
상승 확률: 0.46
하락 확률: 0.54
----------------------------------------
지역 : 서울특별시 중구
예측 결과: 인기 하락
상승 확률: 0.44
하락 확률: 0.56
---------------------------------

---

In [907]:
from sklearn.ensemble import RandomForestClassifier

# RandomForest로 확률 예측
rf = RandomForestClassifier(
    n_estimators=1200,
    min_samples_leaf=2,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

Accuracy: 0.7727272727272727
[[35 10]
 [10 33]]
              precision    recall  f1-score   support

         0.0     0.7778    0.7778    0.7778        45
         1.0     0.7674    0.7674    0.7674        43

    accuracy                         0.7727        88
   macro avg     0.7726    0.7726    0.7726        88
weighted avg     0.7727    0.7727    0.7727        88



In [908]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
y_pred = gb_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

Accuracy: 0.7727272727272727
[[36  9]
 [11 32]]
              precision    recall  f1-score   support

         0.0     0.7660    0.8000    0.7826        45
         1.0     0.7805    0.7442    0.7619        43

    accuracy                         0.7727        88
   macro avg     0.7732    0.7721    0.7723        88
weighted avg     0.7731    0.7727    0.7725        88



In [909]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(
    n_estimators = 200
    , max_depth = 3
    , learning_rate = 0.1
    , random_state = 0
)

xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

Accuracy: 0.8409090909090909
[[38  7]
 [ 7 36]]
              precision    recall  f1-score   support

         0.0     0.8444    0.8444    0.8444        45
         1.0     0.8372    0.8372    0.8372        43

    accuracy                         0.8409        88
   macro avg     0.8408    0.8408    0.8408        88
weighted avg     0.8409    0.8409    0.8409        88



In [910]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgb_clf, X_train, y_train, cv=5)
print(f'원본 데이터 평가 : {np.mean(scores)}')

원본 데이터 평가 : 0.7729606625258798


In [911]:
fi = pd.Series(xgb_clf.feature_importances_, index=feature_cols).sort_values(ascending=False)
fi_df = fi.reset_index()
fi_df.columns = ["feature", "importance"]
fi_df.head(15)

,feature,importance
0,TURSM_SPND_PRICE_MOM_RATE,0.146953
1,BASE_YEAR_ACCMLT_FQ_CO_MOM_RATE,0.091063
2,AVRG_SCORE_VALUE,0.080498
3,TURSM_CSTMR_CO_MOM_RATE,0.075788
4,TURSM_CSTMR_CO_ROLL3,0.048292
5,TURSM_SPND_PRICE_L2,0.048137
6,TURSM_CSTMR_CO_L1,0.039894
7,TURSM_SPND_PRICE_ROLL3,0.039569
8,AVRG_SCORE_VALUE_L1,0.036965
9,REVIEW_CO_MOM_RATE,0.034023


In [912]:
# 모델 저장
from joblib import dump

dump(xgb_clf, 'k_culture_tourism_growth_model.joblib')

['k_culture_tourism_growth_model.joblib']